In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_53437/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_15_9_8,0.993884,0.979896,0.974203,0.980227,0.001144,0.003115,0.002897,0.003012,"Hidden Size=[40], regularizer=0.01, learning_r..."
1,model_15_9_7,0.993881,0.981160,0.975566,0.981381,0.001145,0.002919,0.002744,0.002836,"Hidden Size=[40], regularizer=0.01, learning_r..."
2,model_15_9_9,0.993836,0.978690,0.972795,0.979090,0.001153,0.003301,0.003055,0.003185,"Hidden Size=[40], regularizer=0.01, learning_r..."
3,model_15_9_6,0.993809,0.982471,0.976835,0.982527,0.001158,0.002716,0.002601,0.002662,"Hidden Size=[40], regularizer=0.01, learning_r..."
4,model_15_9_10,0.993752,0.977549,0.971380,0.977985,0.001169,0.003478,0.003214,0.003354,"Hidden Size=[40], regularizer=0.01, learning_r..."
5,model_15_9_5,0.993644,0.983813,0.977946,0.983635,0.001189,0.002508,0.002477,0.002493,"Hidden Size=[40], regularizer=0.01, learning_r..."
6,model_15_9_11,0.993643,0.976477,0.969985,0.976924,0.001189,0.003644,0.003371,0.003515,"Hidden Size=[40], regularizer=0.01, learning_r..."
7,model_15_9_12,0.993517,0.975476,0.968631,0.975915,0.001213,0.003799,0.003523,0.003669,"Hidden Size=[40], regularizer=0.01, learning_r..."
8,model_15_9_13,0.993383,0.974545,0.967331,0.974963,0.001238,0.003943,0.003669,0.003814,"Hidden Size=[40], regularizer=0.01, learning_r..."
9,model_15_9_4,0.993357,0.985163,0.978814,0.984663,0.001243,0.002299,0.002379,0.002337,"Hidden Size=[40], regularizer=0.01, learning_r..."
